In [1]:
from scipy.interpolate import interpn
import numpy as np
from multiprocessing import Pool
from functools import partial
import warnings
import math
warnings.filterwarnings("ignore")

In [39]:
# time line
T_min = 0
T_max = 70
T_R = 45
# discounting factor
beta = 1/(1+0.02)
# utility function parameter 
gamma = 2
# relative importance of housing consumption and non durable consumption 
alpha = 0.8
# parameter used to calculate the housing consumption 
kappa = 0.3
# depreciation parameter 
delta = 0.025
# housing parameter 
chi = 0.3
# uB associated parameter
B = 2
# constant cost 
c_h = 0.5
# All the money amount are denoted in thousand dollars
earningShock = [0.8,1.2]
# Define transition matrix of economical states
# GOOD -> GOOD 0.8, BAD -> BAD 0.6
Ps = np.array([[0.6, 0.4],[0.2, 0.8]])
# current risk free interest rate
# r_b = np.array([0.01 ,0.03]) increase the mean by 2%
r_b = np.array([0.03 ,0.05])
# stock return depends on current and future econ states
# r_k = np.array([[-0.2, 0.15],[-0.15, 0.2]])
r_k = np.array([[-0.2, 0.15],[-0.15, 0.20]])
# expected return on stock market, use to calculate 401k dynamics
# r_bar = 0.0667
r_bar = 0.02
# probability of survival
Pa = np.load("prob.npy")
# deterministic income
detEarning = np.load("detEarning.npy")
# probability of employment transition Pe[s, s_next, e, e_next]
Pe = np.array([[[[0.3, 0.7], [0.1, 0.9]], [[0.25, 0.75], [0.05, 0.95]]],
               [[[0.25, 0.75], [0.05, 0.95]], [[0.2, 0.8], [0.01, 0.99]]]])
# tax rate before and after retirement
tau_L = 0.2
tau_R = 0.1
# constant state variables: Purchase value 250k, down payment 50k, mortgage 200k, interest rate 3.6%,
# 55 payment period, 8.4k per period. One housing unit is roughly 1 square feet. Housing price 0.25k/sf 
# some variables associate with 401k amount
Nt = [np.sum(Pa[t:]) for t in range(T_max-T_min)]
Dt = [np.ceil(((1+r_bar)**N - 1)/(r_bar*(1+r_bar)**N)) for N in Nt]
# income fraction goes into 401k 
yi = 0.005
# mortgate rate 
rh = 0.036
D = [((1+rh)**N - 1)/(rh*(1+rh)**N) for N in range(T_max-T_min)]
D[0] = 1
# housing unit
H = 100
# housing price constant 
pt = 2*250/1000
# 30k rent 1000 sf
pr = 2*30/1000

In [35]:
# M = 100
# tt = 2
# m = M/D[tt-1]
# for i in range(1,tt):
#     M = M*(1+rh) - m
#     print(M)

1.1368683772161603e-13


In [40]:
#Define the utility function
def u(c):
    return (np.float_power(c, 1-gamma) - 1)/(1 - gamma)

#Define the bequeath function, which is a function of wealth
def uB(tb):
    return B*u(tb)

#Calcualte HE 
def calHE(x):
    # the input x is a numpy array 
    # w, n, M, g_lag, e, s = x
    HE = (H+(1-chi)*(1-delta)*x[:,3])*pt - x[:,2]
    return HE

#Calculate TB 
def calTB(x):
    # the input x as a numpy array
    # w, n, M, g_lag, e, s = x
    TB = x[:,0] + x[:,1] + calHE(x)
    return TB

In [43]:
def R(x, a):
    '''
    Input:
        state x: w, n, M, g_lag, e, s
        action a: c, b, k, i, q = a which is a np array
    Output: 
        reward value: the length of return should be equal to the length of a
    '''
    w, n, M, g_lag, e, s = x
    reward = np.zeros(a.shape[0])
    # actions with improvement 
    i_index = (a[:,4]==1)
    # actions without improvement
    ni_index = (a[:,4]!=1)
    # housing consumption with improvement
    i_h = H + (1-delta)*g_lag + a[i_index][:,3]
    i_Vh = (1+kappa)*i_h
    # housing consumption without improvement
    ni_h = H + (1-delta)*g_lag
    ni_Vh = (1-kappa)*(ni_h-(1-a[ni_index][:,4])*H)
    # combined consumption with and without improvement
    i_C = np.float_power(a[i_index][:,0], alpha) * np.float_power(i_Vh, 1-alpha)
    ni_C = np.float_power(a[ni_index][:,0], alpha) * np.float_power(ni_Vh, 1-alpha)
    reward[i_index] = u(i_C)
    reward[ni_index] = u(ni_C)
    return reward

In [44]:
#Define the earning function, which applies for both employment and unemployment, good econ state and bad econ state 
def y(t, x):
    w, n, M, g_lag, e, s = x
    if t <= T_R:
        welfare = 5
        return detEarning[t] * earningShock[int(s)] * e + (1-e) * welfare
    else:
        return detEarning[t]
    
#Earning after tax and fixed by transaction in and out from 401k account 
def yAT(t,x):
    yt = y(t, x)
    w, n, M, g_lag, e, s = x
    if t <= T_R and e == 1:
        # yi portion of the income will be put into the 401k 
        return (1-tau_L)*(yt * (1-yi))
    if t <= T_R and e == 0:
        return yt
    else:
        # t > T_R, n/discounting amount will be withdraw from the 401k 
        return (1-tau_R)*yt + n/Dt[t]

#Define the evolution of the amount in 401k account 
def gn(t, n, x, s_next):
    w, n, M, g_lag, e, s = x
    if t <= T_R and e == 1:
        # if the person is employed, then yi portion of his income goes into 401k 
        n_cur = n + y(t, x) * yi
    elif t <= T_R and e == 0:
        # if the perons is unemployed, then n does not change 
        n_cur = n
    else:
        # t > T_R, n/discounting amount will be withdraw from the 401k 
        n_cur = n - n/Dt[t]
    return (1+r_k[int(s), s_next])*n_cur 

In [ ]:
def transition(x, a, t):
    '''
    Input: state and action and time
    Output: possible future states and corresponding probability 
    '''
    w, n, M, g_lag, e, s = x
    # variables used to collect possible states and probabilities
    x_next = []
    prob_next = []
    # mortgage payment
    m = M/D[T_max-t]
    M_next = M*(1+rh) - m
    for aa in a:
        c,b,k,i,q = aa
        if q == 1:
            g = (1-delta)*g_lag + i
        else:
            g = (1-delta)*g_lag
        for s_next in [0,1]:
            w_next =  b*(1+r_b[int(s)]) + k*(1+r_k[int(s), s_next])
            n_next = gn(t, n, x, s_next)
            if t >= T_R:
                e_next = 0
                x_next.append([w_next, n_next, M_next, g, e_next, s_next])
                prob_next.append(Ps[int(s),s_next])
            else:
                for e_next in [0,1]:
                    x_next.append([w_next, n_next, M_next, g, e_next, s_next])
                    prob_next.append(Ps[int(s),s_next] * Pe[int(s),s_next,int(e),e_next])
    return np.array(x_next), np.array(prob_next)